#DERS 18- LOJİSTİK REGRESYON 

En kolay uygulaması ve açıklaması olan makine öğrenmesi algortimasıdır.. Tahmin etmek istediğimiz değişkenimizin tipi kategorik ise Lojistik Regresyon kullanabiliriz. (Örneğin bir kişinin kanser olup olmadığı, bir kişiye kredi verilip verilmeyeceği veya bir mailin spam olup olmadığı gibi..)

Bu örnekten yola çıkarak  lojistik regresyonun amacını şu şekilde açıklayabiliriz; iki yönlü karakteristiği olan bağımlı değişkenle ilgili bir dizi bağımsız değişken arasında uygun ilişkiyi tanımlamak için en uygun modeli bulmaktır.

 Lojistik regresyon analizinde dikkat edilmesi gereken noktalar;

Bağımsız değişkenler arasında uygun olan tüm değişkenlerin modele dahil edilmesi, böylece model yetersizliğinin önüne geçilebilir.
Bağımsız değişkenler arasından uygun olmayan tüm değişkenlerin modelden dışlanması, böylelikle model karmaşıklıktan kurtulur ve yorumlanması kolay bir hal alır.
Tekrarlanmış ölçümlerden kaçınılmalıdır.
Bağımsız değişkenlerin ölçüm hataları küçük olmalı ve kayıp veri (missing value) olmamalıdır. Hatalar büyük olursa model yanlı ve yetersiz olur.
Bağımsız değişkenler birbiriyle ilişkili olmaması gerekmektedir.
Uç değerler olmamalıdır.
Örneklem sayısı model kurulmasının sağlığı için büyük olmalıdır.

#Telecom Churn Case Study
21 adet değişken yardımıyla müşterinin gelecekte başka bir telecom firmasına geçip geçmeyeceğini logistic regression ile tahminlemeye çalışacağız.





In [ ]:
# Pandas ve Numpy kütüphanelerinin import edilmesi
import pandas as pd
import numpy as np

In [ ]:
# Gerekli veri setlerinin içeriye alınması.
churn_data = pd.read_csv("churn_data.csv")
customer_data = pd.read_csv("customer_data.csv")
internet_data = pd.read_csv("internet_data.csv")

churn_data.head(3)
customer_data.head(3)
internet_data.head(3)

,customerID,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,No,DSL,Yes,Yes,No,No,No,No


In [ ]:
#Veri setlerinin customerID'den inner join ile birleştirilmesi
df_1 = pd.merge(churn_data, customer_data, how='inner', on='customerID')

In [ ]:
telecom = pd.merge(df_1, internet_data, how='inner', on='customerID')

#Veri Setini İnceleme

In [ ]:
# İlk 5 satır, Churn kolonu bizim tahminlememiz gereken bağımlı değişkendir.
telecom.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,Female,0,Yes,No,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No,Male,0,No,No,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Male,0,No,No,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,45,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Male,0,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Female,0,No,No,No,Fiber optic,No,No,No,No,No,No


In [ ]:
telecom.describe() # veri setinin istatistiksel özeti.

,tenure,MonthlyCharges,SeniorCitizen
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,0.162147
std,24.559481,30.090047,0.368612
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,0.000000
50%,29.000000,70.350000,0.000000
75%,55.000000,89.850000,0.000000
max,72.000000,118.750000,1.000000


In [ ]:
# Veri setinin değişken bazında veri tipleri ve özellikleri.
telecom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   tenure            7043 non-null   int64  
 2   PhoneService      7043 non-null   object 
 3   Contract          7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   Churn             7043 non-null   object 
 9   gender            7043 non-null   object 
 10  SeniorCitizen     7043 non-null   int64  
 11  Partner           7043 non-null   object 
 12  Dependents        7043 non-null   object 
 13  MultipleLines     7043 non-null   object 
 14  InternetService   7043 non-null   object 
 15  OnlineSecurity    7043 non-null   object 
 16  OnlineBackup      7043 non-null   object 


#DATA PREP. 

In [ ]:
# Yes ve No değerlerinin 1 ve 0'lara dönüştürülmesi
telecom['PhoneService'] = telecom['PhoneService'].map({'Yes': 1, 'No': 0})
telecom['PaperlessBilling'] = telecom['PaperlessBilling'].map({'Yes': 1, 'No': 0})
telecom['Churn'] = telecom['Churn'].map({'Yes': 1, 'No': 0})
telecom['Partner'] = telecom['Partner'].map({'Yes': 1, 'No': 0})
telecom['Dependents'] = telecom['Dependents'].map({'Yes': 1, 'No': 0})

#Dummy Variable Creation

In [ ]:
# Kategorik değişkenler için dummy variable oluşturulması ve 1 tanesinin veriden çıkartılması.
cont = pd.get_dummies(telecom['Contract'],prefix='Contract',drop_first=True)
telecom = pd.concat([telecom,cont],axis=1)

pm = pd.get_dummies(telecom['PaymentMethod'],prefix='PaymentMethod',drop_first=True)
telecom = pd.concat([telecom,pm],axis=1)


gen = pd.get_dummies(telecom['gender'],prefix='gender',drop_first=True)
telecom = pd.concat([telecom,gen],axis=1)


ml = pd.get_dummies(telecom['MultipleLines'],prefix='MultipleLines')
ml1 = ml.drop(['MultipleLines_No phone service'],1)
telecom = pd.concat([telecom,ml1],axis=1)


iser = pd.get_dummies(telecom['InternetService'],prefix='InternetService',drop_first=True)
telecom = pd.concat([telecom,iser],axis=1)


os = pd.get_dummies(telecom['OnlineSecurity'],prefix='OnlineSecurity')
os1= os.drop(['OnlineSecurity_No internet service'],1)
telecom = pd.concat([telecom,os1],axis=1)


ob =pd.get_dummies(telecom['OnlineBackup'],prefix='OnlineBackup')
ob1 =ob.drop(['OnlineBackup_No internet service'],1)
telecom = pd.concat([telecom,ob1],axis=1)


dp =pd.get_dummies(telecom['DeviceProtection'],prefix='DeviceProtection')
dp1 = dp.drop(['DeviceProtection_No internet service'],1)
telecom = pd.concat([telecom,dp1],axis=1)

 
ts =pd.get_dummies(telecom['TechSupport'],prefix='TechSupport')
ts1 = ts.drop(['TechSupport_No internet service'],1)
telecom = pd.concat([telecom,ts1],axis=1)


st =pd.get_dummies(telecom['StreamingTV'],prefix='StreamingTV')
st1 = st.drop(['StreamingTV_No internet service'],1)
telecom = pd.concat([telecom,st1],axis=1)


sm =pd.get_dummies(telecom['StreamingMovies'],prefix='StreamingMovies')
sm1 = sm.drop(['StreamingMovies_No internet service'],1)
telecom = pd.concat([telecom,sm1],axis=1)

In [ ]:
# Dummy Variable'ları oluşturduğumuz orjinal kolonları çıkartabiliriz.
telecom = telecom.drop(['Contract','PaymentMethod','gender','MultipleLines','InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'], 1)

In [ ]:
#Total Charges alanını string'den float'a çeviriyoruz.
#telecom['TotalCharges']=telecom['TotalCharges'].convert_objects(convert_numeric=True)
pd.to_numeric(telecom['TotalCharges'], errors='coerce')

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

In [ ]:
telecom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 54 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customerID                             7043 non-null   object 
 1   tenure                                 7043 non-null   int64  
 2   PhoneService                           0 non-null      float64
 3   PaperlessBilling                       0 non-null      float64
 4   MonthlyCharges                         7043 non-null   float64
 5   TotalCharges                           7043 non-null   object 
 6   Churn                                  0 non-null      float64
 7   SeniorCitizen                          7043 non-null   int64  
 8   Partner                                0 non-null      float64
 9   Dependents                             0 non-null      float64
 10  Contract_One year                      7043 non-null   uint8  
 11  Cont

In [ ]:
# Checking for outliers in the continuous variables
num_telecom = telecom[['tenure','MonthlyCharges','SeniorCitizen','TotalCharges']]

In [ ]:
# Outlier'ları görebilmek için verinin quartil değerlerini kullanıyoruz.
# görüldüğü üzere sürekli değişkenler gayet normal biçimde artış gösteriyor.
# ve aykırı ya da uç değerler gözükmüyor.
# %99 quartil değerinden sonraki max'değeri birbirine çok yakın bu da outlier olmadığını gösteriyor.
num_telecom.describe(percentiles=[.25,.5,.75,.90,.95,.99])

,tenure,MonthlyCharges,SeniorCitizen
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,0.162147
std,24.559481,30.090047,0.368612
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,0.000000
50%,29.000000,70.350000,0.000000
75%,55.000000,89.850000,0.000000
90%,69.000000,102.600000,1.000000
95%,72.000000,107.400000,1.000000
99%,72.000000,114.729000,1.000000


#BOŞ DEĞER KONTROLÜ 

In [ ]:
# Adding up the missing values (column-wise)
telecom.isnull().sum()

customerID                                  0
tenure                                      0
PhoneService                             7043
PaperlessBilling                         7043
MonthlyCharges                              0
TotalCharges                                0
Churn                                    7043
SeniorCitizen                               0
Partner                                  7043
Dependents                               7043
Contract_One year                           0
Contract_Two year                           0
PaymentMethod_Credit card (automatic)       0
PaymentMethod_Electronic check              0
PaymentMethod_Mailed check                  0
gender_Male                                 0
MultipleLines_No                            0
MultipleLines_Yes                           0
InternetService_Fiber optic                 0
InternetService_No                          0
OnlineSecurity_No                           0
OnlineSecurity_Yes                

In [ ]:
# Yüzdesel olarak değişkenlerin null ya da nan miktarının gösterilmesi.
round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

customerID                                 0.0
tenure                                     0.0
PhoneService                             100.0
PaperlessBilling                         100.0
MonthlyCharges                             0.0
TotalCharges                               0.0
Churn                                    100.0
SeniorCitizen                              0.0
Partner                                  100.0
Dependents                               100.0
Contract_One year                          0.0
Contract_Two year                          0.0
PaymentMethod_Credit card (automatic)      0.0
PaymentMethod_Electronic check             0.0
PaymentMethod_Mailed check                 0.0
gender_Male                                0.0
MultipleLines_No                           0.0
MultipleLines_Yes                          0.0
InternetService_Fiber optic                0.0
InternetService_No                         0.0
OnlineSecurity_No                          0.0
OnlineSecurit

In [ ]:
# kayıp değerler silindikten sonra verinin tekrar kayıp değer durumunun ekrana bastırılması.
round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

customerID                              NaN
tenure                                  NaN
PhoneService                            NaN
PaperlessBilling                        NaN
MonthlyCharges                          NaN
TotalCharges                            NaN
Churn                                   NaN
SeniorCitizen                           NaN
Partner                                 NaN
Dependents                              NaN
Contract_One year                       NaN
Contract_Two year                       NaN
PaymentMethod_Credit card (automatic)   NaN
PaymentMethod_Electronic check          NaN
PaymentMethod_Mailed check              NaN
gender_Male                             NaN
MultipleLines_No                        NaN
MultipleLines_Yes                       NaN
InternetService_Fiber optic             NaN
InternetService_No                      NaN
OnlineSecurity_No                       NaN
OnlineSecurity_Yes                      NaN
OnlineBackup_No                 

In [ ]:
# Sürekli değişkenlerin normalize edilmesi. (Regression olduğu için)
df = telecom[['tenure','MonthlyCharges','TotalCharges']]

normalized_df=(df-df.mean())/df.std()

telecom = telecom.drop(['tenure','MonthlyCharges','TotalCharges'], 1)

telecom = pd.concat([telecom,normalized_df],axis=1)
telecom

,customerID,PhoneService,PaperlessBilling,Churn,SeniorCitizen,Partner,Dependents,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_Male,MultipleLines_No,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_Male,MultipleLines_No,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,tenure,MonthlyCharges,TotalCharges
